# Tensorflow Temelleri ve Doğrusal Regresyon
Bu Jupyter Notebook tensorflow'un temel prensiplerini ve basit bir doğrusal regresyon modelini içermektedir. 
Bu notebook doğrusal regresyonu açıklama amacı gütmez. Bu işlemlerin arkasında yatan matematiği ve mantığı öğrenmek için internetteki birçok doğrusal regresyon kaynağına bakabilirsiniz.

Bu notebook'ta, doğrusal regresyon parametreleri 'Gradient Descent' yöntemiyle öğrenilecektir. En çok kullanılan doğrusal regresyon öğrenim yöntemi sıradan en küçük kareler (ordinary least squares) yöntemi olsa da, ilerideki optimizasyon bölümlerine önayak olması açısından gradient descent kullanmayı uygun gördüm. Siz isterseniz OLS yöntemini kendiniz implement edebilirsiniz.

Bu notebook'taki aşamalar şu şekildedir:

- Libraryleri yükle
- Veriyi yükle
- *Tensorflow işlemlerini ve işlemlerin oluşturduğu graph'ı tanımla*
- *Tensorflow session'ını çalıştır.*

Aynı aşamaları sonrasında Mini-batch Gradient Descent ile tekrarlayıp, graph'ımızı [Tensorboard](https://www.tensorflow.org/get_started/summaries_and_tensorboard) ile görselleştiriyoruz.

In [1]:
# Libraryleri yükle
import numpy as np
from sklearn.datasets import load_boston
import tensorflow as tf # RuntimeWarning çıkarsa sorun değil
import pandas as pd
tf.__version__

/Users/elvan/anaconda3/envs/tensorflow/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


'1.4.1'

In [2]:
# Veriyi oku
boston = load_boston() # boston.data ve boston.target olmak üzere iki array içerir
m, n = boston.data.shape # verinin satır sütun sayıları
print(m, n)
boston_plus_bias = np.c_[np.ones((m, 1)), boston.data]
pd.DataFrame(boston_plus_bias).head()

506 13


,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,1.0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,1.0,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,1.0,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,1.0,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,1.0,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33


Görülüğü üzere, veriyi ilk yüklediğimizde 506 satır, 13 sütunluk bir verisetine sahiptik. 

Ancak doğrusal regresyon ve yapay sinir ağları modelleri için bir 'bias' gerekmektedir. 'Bias' dediğimiz kısım, sabit bir değer olup, doğrusal regresyon için 'intercept' değerine denk gelmektedir. Bu sabit değeri öğrenebilmek için verimize 1 sayısından oluşan bir sütun ekliyoruz. Bu sütuna denk gelecek ağırlık bizim 'bias' değerimiz olacaktır.

Bu noktadan sonra, verimize eriştiğimize göre Tensorflow değişkenlerini tanımaya ve graph'ımızı tanımlamaya geçebiliriz

## Tensorflow'un Çalışma Mantığı

Tensorflow yapısı gereği önce değişkenleri ve değişkenlere yapılacak işlemleri tanımlamanızı gerektirir. Sonrasında ise bu işlemleri koşturarak bir sonuç üretir. Bu aşamaların sebebi, tensorflow'un öncelikle bir koşturma grafiği oluşturmasıyla alakalıdır. Bu grafik sayesinde Tensorflow, grafiğin parçalarını paralel çalıştırarak hız ve performans kazanır. 

Bu yöntemin dezavantajı ise interaktivite kaybıdır. Normal Python kodları gibi interaktif çalışmak problemli olabilir. Bunun için üretilmiş daha üst seviye kütüphaneler mevcuttur. En çok bilineni [Keras](https://keras.io/) olan bu kütüphanelerin bir benzerini de Google [Tensorflow Eager](https://github.com/tensorflow/tensorflow/tree/master/tensorflow/contrib/eager) adıyla çıkarmıştır ancak benim henüz bu kütüphaneyi deneme fırsatım olmadı. Daha basit bir çalışma süreci isterseniz Keras kullanabilirsiniz ancak bu notebooklar ana Tensorflow'a odaklanmaktadır.

In [3]:
# Değişkenleri tanımlar
X = tf.constant(boston_plus_bias, dtype = tf.float32, name = 'X')
y = tf.constant(boston.target.reshape(-1,1), dtype=tf.float32, name = "y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1, 1), name = "theta")

Tanımladığımız değişkenlerden X, y sabit; theta ise değişkendir (Variable). Bunun sebebi, verimiz olan X ve y'nin değişmez olması; öğrenmek istediğimiz değişken olan theta'nın ise kodun çalışma süresi boyunca değişecek olmasıdır.

Şimdi öğrenme parametrelerini tanımlayalım.

In [4]:
n_epochs = 2000 #2000 iterasyon. 
# Kendi bilgisayarınıza göre düşürüp arttırabilirsiniz.

learning_rate = 0.000001 # Gradient descent adım boyutu

Bu parametreler çoğunlukla deneme yanılma yöntemi ve öğrenme performansına bakılarak belirlenir. Eğer Gradient Descent yöntemini bilmiyorsanız öğrenmek için Fast.ai [wiki](http://wiki.fast.ai/index.php/Gradient_Descent)'sine bakmanızı tavsiye ederim.

Bu noktadan sonra yapmamız gereken şey, Tensorflow'un yapacağı işlemleri tanımlamak.

In [5]:
y_pred = tf.matmul(X, theta, name = "predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name = "mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

Bu noktada, Tensorflow'un yapacağı işlemleri tanımladık ancak henüz hiçbir işlem yapılmadı. 

İlk satırda, tensorflow'a tahminleri nasıl hesaplayacağını söylüyoruz.

İkinci satırda, hatayı nasıl hesaplayacağını söylüyoruz.

Üçüncü satırda tüm kare hataların ortalamasını nasıl alacağını söylüyoruz.

Dördüncü satırda, Gradient Descent optimizasyonunu kullanmasını ve bu optimizasyonun adım boyutunu belirtiyoruz.

Beşinci satırda optimizasyonun, ortalama kare hatayı minimize ederek yapılacağını söylüyoruz.

İşlemleri gerçekleştirmek için öncelikle bir Tensorflow session'ı oluşturmamız gerekiyor. Bu session graph'ı oluşturarak işlemleri koşturacak ve bu sayede sonuçları elde edebileceğiz. 

Henüz ne değişkenler ne de işlemler başlatılmadı. İlk yapacağımız şey değişkenleri başlatıp, sonrasında işlemleri koşturmak.

In [6]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init) # Değişkenleri başlat
    
    # Optimizasyonu epoch sayısı kadar çalıştır
    for epoch in range(n_epochs):
        if epoch % 100 == 0: # Her 100 epoch'te bir performans çıktısı
            print("Epoch", epoch, "MSE = ", mse.eval())
        sess.run(training_op)
    
    best_theta = theta.eval()

Epoch 0 MSE =  227182.0
Epoch 100 MSE =  583.323
Epoch 200 MSE =  518.257
Epoch 300 MSE =  464.711
Epoch 400 MSE =  420.195
Epoch 500 MSE =  382.842
Epoch 600 MSE =  351.227
Epoch 700 MSE =  324.259
Epoch 800 MSE =  301.089
Epoch 900 MSE =  281.055
Epoch 1000 MSE =  263.632
Epoch 1100 MSE =  248.4
Epoch 1200 MSE =  235.021
Epoch 1300 MSE =  223.22
Epoch 1400 MSE =  212.769
Epoch 1500 MSE =  203.48
Epoch 1600 MSE =  195.195
Epoch 1700 MSE =  187.782
Epoch 1800 MSE =  181.127
Epoch 1900 MSE =  175.135


Gradient descent söz konusu olduğu zaman dikkat edilmesi gereken en büyük nokta öğrenme adımının boyutudur. Eğer öğrenme adımınız çok büyükse 'Exploding weights' (büyüyen ağırlıklar) problemiyle karşılaşırsınız. Bu durumda, öğrenme sürecinde hatanız azalmak yerine sonsuza gidecektir. İsterseniz bu notebook'te learning_rate'i değiştirerek deneyebilirsiniz.

## Minibatch Gradient Descent

Bir önceki optimizasyon yönteminin ufak bir sorunu var: Parametreleri öğrenebilmek için bütün veriyi kullanıyor. Bu kadar ufak bir veri için bu bir sorun değil. Ancak ileride, RAM'de tutamayacak kadar büyük bir veri olduğu durumda, veriyi parça parça okuyarak işlemleri gerçekleştirmek gerekiyor. Bu yüzden kodu biraz değiştirmemiz gerek.

Tensorflow'a her bir batch iterasyonunda yeni okuduğumuz verileri iletebilmek için 'placeholder' adı verilen değişkenler oluşturmamız gerekiyor. Aslında her tensorflow değişkenine yeni veri iletebilirsiniz ancak en doğru yolu placeholderları kullanmak. Placeholderlar kodu veri olmadan koşturmaya çalıştığınızda hata verir, bu sayede veri beslemeyi unutmazsınız.

Hazır kodu değiştirmişken graph'ımızı görselleştirmek için bir loglama prosedürü de ekleyelim. Sonrasında Tensorboard ile performans değişimine bakabiliriz.

Graph'ın kalabalıklaşmaması için graph'ı öncelikle resetliyoruz.

In [7]:
tf.reset_default_graph()
X = tf.placeholder(tf.float32, shape = (None, n+1), name = "X")
y = tf.placeholder(tf.float32, shape = (None, 1), name = "y")

# Parça boyutu ve sayısı
batch_size = 100
n_batches = int(np.ceil(m/batch_size)) # Python'un int-float kurallarına dikkat

# Parçaları çağırmak için fonksiyonu tanımla
def fetch_batch(epoch, batch_index, batch_size):
    np.random.seed(epoch * n_batches + batch_index)  
    indices = np.random.randint(m, size=batch_size)  
    X_batch = boston_plus_bias[indices] # Buradaki veri değişebilir
    y_batch = boston.target.reshape(-1, 1)[indices] 
    y_batch = y_batch.reshape(-1,1)
    return X_batch, y_batch

Loglama için gerekli kütüphaneleri yükleyip, değişkenleri oluşturuyoruz.

In [8]:
from datetime import datetime
now = datetime.utcnow().strftime("%Y%m%d&H%M%S")
root_logdir = "tf_logs_lr"
logdir = "{}/run-{}".format(root_logdir, now)
n_epochs = 2000 
learning_rate = 0.000001 # Gradient descent adım boyutu

In [9]:
for batch_index in range(n_batches):
    X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
    print(X_batch.shape)     

(100, 14)
(100, 14)
(100, 14)
(100, 14)
(100, 14)
(100, 14)


In [10]:
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1, 1), name = "theta")
y_pred = tf.matmul(X, theta, name = "predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name = "mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)
init = tf.global_variables_initializer()

# Loglama bilgilerinin değişkenlerini oluştur
mse_summary = tf.summary.scalar('MSE', mse)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

with tf.Session() as sess:
    sess.run(init) # Değişkenleri başlat
    
    # Optimizasyonu epoch sayısı kadar çalıştır
    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            if(batch_index % 10 == 0):
                summary_str = mse_summary.eval(feed_dict = {X: X_batch, y:y_batch})
                step = epoch*n_batches + batch_index
                file_writer.add_summary(summary_str, step)
        if epoch % 100 == 0: # Her 100 epoch'te bir performans çıktısı
            print("Epoch", epoch, "MSE = ", mse.eval(feed_dict={X: X_batch, y:y_batch}))
            sess.run(training_op, feed_dict={X: X_batch, y:y_batch})
    file_writer.close()
    best_theta = theta.eval()

Epoch 0 MSE =  559260.0
Epoch 100 MSE =  58005.2
Epoch 200 MSE =  9470.13
Epoch 300 MSE =  1473.75
Epoch 400 MSE =  680.29
Epoch 500 MSE =  737.706
Epoch 600 MSE =  771.147
Epoch 700 MSE =  474.938
Epoch 800 MSE =  632.57
Epoch 900 MSE =  688.002
Epoch 1000 MSE =  672.85
Epoch 1100 MSE =  622.763
Epoch 1200 MSE =  526.597
Epoch 1300 MSE =  519.646
Epoch 1400 MSE =  617.421
Epoch 1500 MSE =  526.324
Epoch 1600 MSE =  543.233
Epoch 1700 MSE =  628.899
Epoch 1800 MSE =  583.758
Epoch 1900 MSE =  403.859


Bu noktada /tf_logs_lr/ klasörüne bir 'run' ile başlayan bir klasör oluşturulmuş olması gerekiyor. Bu klasörün içindeki dosya bizim öğrenme sürecimizi ve graph'ımızı içeriyor. Bu graph'ı aşağıdaki kodla Tensorboard'a atarak inceleyebilirsiniz.

Tensorboard'u aç:
```shell
tensorboard --logdir tf_logs_lr/
```

Sonrasında ise http://0.0.0.0:6006 adresine girerek Tensorboard'a ulaşabilirsiniz.

Benim makinemdeki Tensorboard'ın görselleri aşağıdaki gibi.

MSE: ![alt text](images/tb_scalar.png "MSE")

Graph: ![alt text](images/tb_graph.png "Graph")